In [1]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorboard.plugins import projector


In [ ]:
from ipynb.fs.full._01_Data_Augmentation import augment

from ipynb.fs.full._02_Train_Val_Test import create_speaker_DF
from ipynb.fs.full._02_Train_Val_Test import choose

from ipynb.fs.full._03_Data import create_data
from ipynb.fs.full._03_Data  import getAbsoluteCounts

from ipynb.fs.full._04_Model import do_all

from ipynb.fs.full._05_Eval import boxplot
from ipynb.fs.full._05_Eval import speakerFalse
from ipynb.fs.full._05_Eval import confusionMatrix
from ipynb.fs.full._05_Eval import weighted_f1
from ipynb.fs.full._05_Eval import combined_scores

from ipynb.fs.full._06_Compare import boxplot_diff
from ipynb.fs.full._06_Compare import significant
from ipynb.fs.full._06_Compare import raincloud_diff
from ipynb.fs.full._06_Compare import compareConfusionMatrix


## Variables

In [3]:
# general Path of all Audios
general_path = "..."
# name of folder inside general_pah for Audios
name = '...'
data_path = general_path + name
# name of Augmentation
# Keywords: 'best'->SeR and FM, 'shifting_pitch', 'segment_removal', 'background_noise'
# 'segment_swap', 'volume_confusion', 'time_reversing', 'time_stretching', 'speed_confusion', 'time_masking'
# 'frequency_masking', 'frequency_swap', 'frequency_insertion', 'speaker_insertion'
name_aug = ''
data_path_aug = general_path + name_aug
# name of baseline to compare with (there should be a pkl with the name './Results_' + name_base)
name_base = ''


# number of augmented files per original
aug_num = 6
# percentage for augmentation per file
aug_perc = 1.0
# segment length for each augmentation
aug_len = 1.0

# length of one segment
audio_length = 10.0
# what are the maximal seconds for one speaker/dialect (must be a multiple of audio_length or None, can't be used with augmentation)
max_length_speaker = None #x*audio_length
max_length_dialect = None #y*audio_length

#model path for extracting embeddings
model_path = "C:\\Users\\Lea\\Desktop\\Doktor\\Models\\trillsson4"
# learning rate
lr = 0.0005
# dropout rate
dr = 0.3
# units dense layer
units = 128
# size of one batch for trillsson Model
batch_size_embedding = 10
# size of one batch for CNN
batch_size = 1024
# L1 regularization parameter used in the dense layers
l1 = 0.013
# L2 regularization parameter used in the dense layers
l2 = 0.0026
# maximal number of epochs
max_epochs = 250

# number of individual runs
runs = 250
# pictures of first Epoch
first_pictures = True
# if not None Speaker for Test and Val will not be random
# give a List with List like [['OLALT3', 'HBALT3', 'FLALT2', 'CUXALT4'], ['BLALT1', 'BBALT2', 'CWALT', 'ULALT1']]
# each List represents an dialect and the first half of each sublist the Speaker for Testing and the second half the speaker for Validation
test = None #[['...', '...'], ['...', '...', '...', '...'], ...]
# test with dummy-classificator
dummy = False
# project to TB
tb = True
log_dir = '...'
# Test different Hyperparameters
hyper_test = False


## Augmentation

In [4]:
if (name_aug != ''):
    augment(name_aug, data_path, data_path_aug, aug_perc, aug_num, aug_len, audio_length, test)
    

## DF with all Audios

In [5]:
df_speaker = create_speaker_DF(data_path, data_path_aug, name_aug)
df_speaker = pd.read_pickle('./All_Files_.pkl')


In [ ]:
df_speaker


## Extract Embeddings

In [ ]:
df_learn = create_data(model_path, audio_length, batch_size_embedding, '', None, None)
if (name_aug != ''):
    df_learn_aug = create_data(model_path, audio_length, batch_size_embedding, name_aug, max_length_speaker, max_length_dialect)

df_learn = pd.read_pickle('./Data_.pkl')
if (name_aug != ''):
    df_learn_aug = pd.read_pickle('./Data_' + name_aug + '_aug.pkl')
    

In [ ]:
df_learn

In [ ]:
if (name_aug != ''):
    print(df_learn_aug)

In [ ]:
getAbsoluteCounts(None)
if (name_aug != ''):
    getAbsoluteCounts(name_aug)

### Projection of embeddings to TB

In [11]:
if tb:
    labels1 = np.array(df_learn['dialect'].tolist())
    len_1 = len(labels1)
    labels2 = np.array(df_learn['speaker'].tolist())
    embeddings = np.array(df_learn['trillsson'].values.tolist())
    if (name_aug != ''):
            labels3 = np.array(df_learn_aug['dialect'].tolist())
            labels4 = np.array(df_learn_aug['speaker'].tolist())
            embeddings2 = np.array(df_learn_aug['trillsson'].values.tolist())
            labels1 = np.concatenate((labels1, labels3), axis=None)
            labels2 = np.concatenate((labels2, labels4), axis=None)
            embeddings = np.concatenate((embeddings, embeddings2), axis=0)

    with open(os.path.join(log_dir, 'metadata.tsv'), "w") as metadata:
        for i in range(0, len(labels1)):
            if i >= len_1:
                metadata.write(f'{labels1[i] + "_" + labels2[i] + "_aug"}\n')
            else:
                metadata.write(f'{labels1[i] + "_" + labels2[i]}\n')
        
    embeddings_tensor = tf.Variable(embeddings)

    checkpoint = tf.train.Checkpoint(embedding=embeddings_tensor)
    checkpoint.save(os.path.join(log_dir, "embedding.ckpt"))

    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
    embedding.metadata_path = 'metadata.tsv'
    projector.visualize_embeddings(log_dir, config)


## Test

In [12]:
df_learned = pd.DataFrame(columns=['acc_train', 'acc_val', 'acc_test',
                                    'loss_train', 'loss_val', 'loss_test',
                                    'f1_train', 'f1_val', 'f1_test',
                                    'f1_w_train', 'f1_w_val', 'f1_w_test',
                                    'speaker_val', 'speaker_test', 'false_names', 'false_speaker', 'false_segments_begin',
                                    'false_segments_end', 'false_simplified', 'classes_x', 'classes_true', 'y_test_names', 
                                    'y_test_speaker'])


In [ ]:
for i in range(0, runs):
    print('Iteration ', i, ' of ' + str(runs))
    
    if test is None:
        test_val = choose(df_speaker)
        print(test_val)
    else:
        test_val = test

    if (hyper_test):
        lr_test = random.uniform(lr[0], lr[1])
        dr_test = random.uniform(dr[0], dr[1])
        l1_test = random.uniform(l1[0], l1[1])
        l2_test = random.uniform(l2[0], l2[1])
        units_test_num = random.randint(0, len(units)-1)
        units_test = units[units_test_num]
        batch_size_test_num = random.randint(0, len(batch_size)-1)
        batch_size_test = batch_size[batch_size_test_num]
             
    if (hyper_test and df_learn_aug is not None):
        list_row, label_mapping = do_all(first_pictures, df_learn, df_learn_aug, test_val, name_aug, i, lr_test, dr_test, units_test, l1_test_conv, l2_test_conv, l1_test, l2_test, batch_size_test, tb, log_dir, dummy, max_epochs)  
    elif (df_learn_aug is not None):
        list_row, label_mapping = do_all(first_pictures, df_learn, df_learn_aug, test_val, name_aug, i, lr, dr, units, l1, l2, batch_size, tb, log_dir, dummy, max_epochs)   
    elif (hyper_test):
        list_row, label_mapping = do_all(first_pictures, df_learn, None, test_val, name_aug, i, lr_test, dr_test, units_test, l1_test_conv, l2_test_conv, l1_test, l2_test, batch_size_test, tb, log_dir, dummy, max_epochs)  
    else:
        list_row, label_mapping = do_all(first_pictures, df_learn, None, test_val, name_aug, i, lr, dr, units, l1, l2, batch_size, tb, log_dir, dummy, max_epochs)
    df_learned.loc[len(df_learned)] = list_row
    

In [14]:
df_learned.to_pickle('./Results_' + name + '_' + name_aug + '.pkl')
df_learned.to_csv('./Results_' + name + '_' + name_aug + '.csv',  sep=';')


## Evaluation

In [ ]:
boxplot('f1_test', name, name_aug)

In [ ]:
boxplot('f1_w_test', name, name_aug)

In [ ]:
boxplot('acc_test', name, name_aug)

In [ ]:
weighted_f1(name, name_aug, label_mapping)

In [19]:
speakerFalse(name, name_aug)

In [ ]:
confusionMatrix(name, name_aug, label_mapping, True)
confusionMatrix(name, name_aug, label_mapping, False)


In [21]:
if (name_base is not None):
    compareConfusionMatrix(name, name_aug, name_base, label_mapping, True)
    compareConfusionMatrix(name, name_aug, name_base, label_mapping, False)
    

In [22]:
if (name_base is not None):
    boxplot_diff('f1_w_test', name_base, name, name_aug)
    raincloud_diff('f1_w_test', name_base, name, name_aug)
    

In [ ]:
#compares results base model with new model (two-sided)
if (name_base is not None):
    U_base, U_aug, p_U, res_T, var_base, var_aug = significant('./Results_' + name_base + '.pkl', './Results_' + name + '_' + name_aug + '.pkl', 'f1_w_test', 'two-sided')
    print("two-sided U-test: ", f'{p_U:.20f}')
    print("two-sided T-test: ", f'{res_T.pvalue:.20f}')
    

In [ ]:
#tests whether results from base model is less (worse) than from new model
if (name_base is not None):
    U_base, U_aug, p_U, res_T, var_base, var_aug = significant('./Results_' + name_base + '.pkl', './Results_' + name + '_' + name_aug + '.pkl', 'f1_w_test', 'less')
    print("one-sided U-test: ", f'{p_U:.20f}')
    print("two-sided T-test: ", f'{res_T.pvalue:.20f}')
    

In [ ]:
combined_scores(name, name_aug)


In [ ]:
confusionMatrix(name, name_aug + "_combined", label_mapping, True)
confusionMatrix(name, name_aug + "_combined", label_mapping, False)
